In [1]:
import numpy as np
from my_ueg import my_ueg
from pyscf import gto, scf, cc

In [2]:
nocc = 1 # 7, 19, 27, 33, 57, 81, 93
my_sys = my_ueg(rs=1.0, nelec=(nocc, nocc), gamma=2) #np.sqrt(2))
print('num kpts: ',my_sys.nkpts)
h0 = my_sys.madelung() / 2
h1 = my_sys.get_h1_real()
cderi = my_sys.get_cderi_real()

mol = gto.M()
mol.nelectron = my_sys.nparticle
mol.nao = my_sys.nkpts
mol.incore_anyway = True
mol.max_memory = 20000
mol.verbose = 4

dm = np.zeros((my_sys.nkpts,my_sys.nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0

mf = scf.RHF(mol).density_fit()
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(my_sys.nkpts)
mf._cderi = cderi
mf.init_guess = "1e"
mf.max_cycle = -1
escf = mf.kernel(dm0=dm)

mf.mo_coeff = np.eye(my_sys.nkpts)

print(f"escf: {escf}")
print(f"escf per electron: {escf/my_sys.nparticle}")

mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
mycc.conv_tol = 1e-7
mycc.kernel()

et = mycc.ccsd_t()
print("CCSD energy", mycc.e_corr)
print("CCSD energy per electron", mycc.e_corr / my_sys.nparticle)
print("CCSD(T) energy", et)
print("CCSD(T) energy per electron", et / my_sys.nparticle)

num kpts:  7


******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmp3rs8wz0o
max_memory 20000 MB (current use 119 MB)
Set gradient conv threshold to 3.16228e-05
init E= -1.39700704811961
  HOMO = 0  LUMO = 4.62866296240225
SCF not converged.
SCF energy = -1.39700704811961
escf: -1.3970070481196089
escf per electron: -0.6985035240598044

******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 1, nmo = 7
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 120 MB)
Init t2, MP2 energy = -1.41292743243707  E_corr(MP2) -0.0159203843174619
Init E_corr(RCCSD) = -0.0159203843174619
cycle = 1

Overwritten attributes  energy_nuc get_hcore get_ovlp  of <class 'pyscf.df.df_jk.DFRHF'>


cycle = 2  E_corr(RCCSD) = -0.0148357684393861  dE = -8.82785517e-05  norm(t1,t2) = 0.000229951
cycle = 3  E_corr(RCCSD) = -0.0148295981364173  dE = 6.17030297e-06  norm(t1,t2) = 1.72805e-05
cycle = 4  E_corr(RCCSD) = -0.0148295981601261  dE = -2.3708837e-11  norm(t1,t2) = 1.42557e-10
RCCSD converged
E(RCCSD) = -1.411836646279735  E_corr = -0.01482959816012612
RCCSD(T) correction = 0
CCSD energy -0.014829598160126122
CCSD energy per electron -0.007414799080063061
CCSD(T) energy 0.0
CCSD(T) energy per electron 0.0


In [5]:
print(my_sys.kpts)

[[0. 0. 0.]
 [0. 0. 0.]]


In [4]:
my_sys.prep_afqmc(mycc)

# Size of the correlation space:
# Number of electrons: (7, 7)
# Number of basis (k-points): 19
# Number of CholVecs (q-points): 92
